In [1]:
import keras
from keras.models import load_model
from keras.datasets import cifar10
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization, Activation, Flatten, Dropout, GlobalAveragePooling2D
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam
from keras.utils import np_utils
import os
import pickle
import numpy as np
import h5py
from keras import backend as K
from keras.optimizers import SGD
from keras.callbacks import LearningRateScheduler

Using TensorFlow backend.


In [2]:
train=np.load("/mnt/Doc/512pix tissue all/npy/512pix_train_1000_picture.npy")
train_label=np.load("/mnt/Doc/512pix tissue all/npy/512pix_train_1000_type.npy")
#test=np.load("/mnt/Doc/512pix_to_128pix/npy/128pix_testpicture.npy")
#test_label=np.load("/mnt/Doc/512pix_to_128pix/npy/128pix_testtype.npy")
val=np.load("/mnt/Doc/512pix tissue all/npy/512pix_val_200_picture.npy")
val_label=np.load("/mnt/Doc/512pix tissue all/npy/512pix_val_200_type.npy")

x_train = train.astype('float32')
#x_test = test.astype('float32')
x_val = val.astype('float32')
x_train /= 255
#x_test /= 255
x_val /=255
del train 
#del test
del val

In [3]:
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print(x_val.shape[0], 'val samples')

x_train shape: (9000, 128, 128, 3)
9000 train samples
1500 test samples
1500 val samples


In [5]:
# divide x_test into validation and test
#x_val = x_test[:900]
#x_test = x_test[900:]
#x_val_label = test_label[:900]
#x_test_label = test_label[900:]


In [6]:
#print("validation data: {0} \ntest data: {1}".format(x_val.shape, x_test.shape))

In [3]:
train_label_1=np_utils.to_categorical(train_label)
#test_label_1=np_utils.to_categorical(test_label)
val_label_1=np_utils.to_categorical(val_label)


In [4]:
batch_size = 16
num_classes = 10
epochs = 20

In [5]:
#重みの読み込み→ボトルネック特徴量の抽出
input_img = Input(shape=(512, 512, 3))
x = Conv2D(16, (7, 7), padding='same')(input_img)
#x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(32, (5, 5), padding='same')(x)
#x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(64, (3, 3), padding='same')(x)
#x = BatchNormalization()(x)
x = Activation('relu')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)
model = Model(input_img,encoded)

weights_path = '/mnt/Doc/512pix_to_128pix/best_weight_0220.h5' 
assert os.path.exists(weights_path), 'Model weights not found '
f = h5py.File(weights_path)
layer_names = [n.decode('utf8') for n in f.attrs['layer_names']]
weight_value_tuples = []
for k, name in enumerate(layer_names):
    if k >= len(model.layers):
        # 全結合層の重みは読み込まない
        break
    g = f[name]
    weight_names = [n.decode('utf8') for n in g.attrs['weight_names']]
    if len(weight_names):
        weight_values = [g[weight_name] for weight_name in weight_names]
        layer = model.layers[k]
        symbolic_weights = layer.trainable_weights + layer.non_trainable_weights
        if len(weight_values) != len(symbolic_weights):
            raise Exception('Layer #' + str(k) +
                            ' (named "' + layer.name +
                            '" in the current model) was found to '
                            'correspond to layer ' + name +
                            ' in the save file. '
                            'However the new layer ' + layer.name +
                            ' expects ' + str(len(symbolic_weights)) +
                            ' weights, but the saved weights have ' +
                            str(len(weight_values)) +
                            ' elements.')
        weight_value_tuples += zip(symbolic_weights, weight_values)
K.batch_set_value(weight_value_tuples)
f.close()
print('Model loaded.')

bottleneck_features_train = model.predict(x_train)
pass_ ="/mnt/Doc/512pix tissue all/bottleneck"
pass_train = os.path.join(pass_,"bottleneck_train.npy")
np.save(pass_train,bottleneck_features_train)

pass_val = os.path.join(pass_,"bottleneck_val.npy")
bottleneck_features_val = model.predict(x_val)
np.save(pass_val,bottleneck_features_val)


Model loaded.


In [3]:
weights_path = '/mnt/Doc/512pix_to_128pix/best_weight_0220.h5' 
f = h5py.File(weights_path)
f 

<HDF5 file "best_weight_0220.h5" (mode r+)>

In [6]:
#全結合層のみ学習==これだめだったは....
def train_top_model():
    
    nb_epoch = 300
    
    train_data =np.load("/mnt/Doc/512pix tissue all/bottleneck/bottleneck_train.npy")
    val_data = np.load("/mnt/Doc/512pix tissue all/bottleneck/bottleneck_val.npy")
    train_label_1=np_utils.to_categorical(train_label)
    val_label_1=np_utils.to_categorical(val_label)

    
    input_img = Input(shape=(train_data.shape[1:]))
    x = Flatten()(input_img)
    x = Dense(512)(x)
    x = Activation("relu")(x)
    x = Dropout(0.5)(x)
    x = Dense(3, activation = "softmax")(x)
    
    model = Model(input_img, x)
    model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=["accuracy"])
    hist = model.fit(train_data,train_label_1,
                    nb_epoch=nb_epoch,
                    validation_data=(val_data,val_label_1),
                    )
    

In [7]:
train_top_model()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:23: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 999 samples, validate on 201 samples
Epoch 1/300


ResourceExhaustedError: OOM when allocating tensor with shape[262144,512] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node training/Adam/mul_1}} = Mul[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"](Adam/beta_1/read, training/Adam/Variable/read)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


In [25]:
#weights_path = '/mnt/Doc//Autoencoder_data/2019.02.15/weight2019.02.15 03:19:21.h5'


input_img = Input(shape=(128, 128, 3))
x = Conv2D(16, (7, 7), padding='same')(input_img)
#x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(32, (5, 5), padding='same')(x)
#x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(64, (3, 3), padding='same')(x)
#x = BatchNormalization()(x)
x = Activation('relu')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)
x = Flatten()(encoded)
Softmax = Dense(3, activation = "softmax")(x)

model = Model(input_img,Softmax)

assert os.path.exists(weights_path), 'Model weights not found '
f = h5py.File(weights_path)
layer_names = [n.decode('utf8') for n in f.attrs['layer_names']]
weight_value_tuples = []
for k, name in enumerate(layer_names):
    if k >= 10:
        # Flattenより上の重みを読み込む
        break
    g = f[name]
    weight_names = [n.decode('utf8') for n in g.attrs['weight_names']]
    if len(weight_names):
        weight_values = [g[weight_name] for weight_name in weight_names]
        layer = model.layers[k]
        symbolic_weights = layer.trainable_weights + layer.non_trainable_weights
        if len(weight_values) != len(symbolic_weights):
            raise Exception('Layer #' + str(k) +
                            ' (named "' + layer.name +
                            '" in the current model) was found to '
                            'correspond to layer ' + name +
                            ' in the save file. '
                            'However the new layer ' + layer.name +
                            ' expects ' + str(len(symbolic_weights)) +
                            ' weights, but the saved weights have ' +
                            str(len(weight_values)) +
                            ' elements.')
        weight_value_tuples += zip(symbolic_weights, weight_values)
K.batch_set_value(weight_value_tuples)
f.close()
print('Model loaded.')

model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=["accuracy"])
hist = model.fit(x_train,train_label_1,
                 batch_size=64,
                    nb_epoch=300,
                    validation_data=(x_val,val_label_1),
                    shuffle=True)


Model loaded.


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:57: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 9000 samples, validate on 1500 samples
Epoch 1/300
9000/9000 [==============================] - 17s 2ms/step - loss: 1.1067 - acc: 0.3339 - val_loss: 1.0987 - val_acc: 0.3513
Epoch 2/300
9000/9000 [==============================] - 16s 2ms/step - loss: 1.0987 - acc: 0.3323 - val_loss: 1.0983 - val_acc: 0.3667
Epoch 3/300
2240/9000 [======>.......................] - ETA: 12s - loss: 1.0978 - acc: 0.3415

KeyboardInterrupt: 

In [ ]:
x = Flatten()(encoded)
x = Dense(128)(x)
x = Activation("relu")(x)
x = Dense(3)(x)
out = Activation("softmax")(x

In [5]:
train=np.load("/mnt/Doc/kazuki-de/2048pix_tissue_all/train_picture.npy")
train_label=np.load("/mnt/Doc/kazuki-de/2048pix_tissue_all/train_type.npy")
#test=np.load("/mnt/Doc/512pix_to_128pix/npy/128pix_testpicture.npy")
#test_label=np.load("/mnt/Doc/512pix_to_128pix/npy/128pix_testtype.npy")
val=np.load("/mnt/Doc/kazuki-de/2048pix_tissue_all/val_picture.npy")
val_label=np.load("/mnt/Doc/kazuki-de/2048pix_tissue_all/val_type.npy")


#x_test = test.astype('float32')
#del train

#del val
train /= 255
#x_test /= 255
val /=255

#del test

train_label_1=np_utils.to_categorical(train_label)
#test_label_1=np_utils.to_categorical(test_label)
val_label_1=np_utils.to_categorical(val_label)


In [13]:
input_img = Input(shape=(299, 299, 3))
x = Conv2D(16, (7, 7), padding='same')(input_img)
x = Activation('relu')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(32, (5, 5), padding='same')(x)
x = Activation('relu')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(64, (3, 3), padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = GlobalAveragePooling2D()(x)
x = Dense(50, activation='relu')(x)
predictions = Dense(3, activation='softmax')(x)
model = Model(inputs=input_img, outputs=predictions)
#for layer in base_model.layers:
#    layer.trainable = False

model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=["accuracy"])
hist = model.fit(train,train_label_1,
                 batch_size=16,
                    nb_epoch=100,
                    validation_data=(val,val_label_1),
                    shuffle=True)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:23: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 7000 samples, validate on 400 samples
Epoch 1/100
7000/7000 [==============================] - 70s 10ms/step - loss: 1.0536 - acc: 0.4534 - val_loss: 2.4088 - val_acc: 0.2800
Epoch 2/100
7000/7000 [==============================] - 70s 10ms/step - loss: 1.0326 - acc: 0.4799 - val_loss: 1.4696 - val_acc: 0.5800
Epoch 3/100
7000/7000 [==============================] - 70s 10ms/step - loss: 1.0226 - acc: 0.4917 - val_loss: 1.5139 - val_acc: 0.1650
Epoch 4/100
7000/7000 [==============================] - 70s 10ms/step - loss: 1.0118 - acc: 0.4976 - val_loss: 1.3708 - val_acc: 0.5800
Epoch 5/100
7000/7000 [==============================] - 70s 10ms/step - loss: 1.0052 - acc: 0.5029 - val_loss: 1.9449 - val_acc: 0.5800
Epoch 6/100
7000/7000 [==============================] - 70s 10ms/step - loss: 0.9973 - acc: 0.5163 - val_loss: 4.2333 - val_acc: 0.2800
Epoch 7/100
7000/7000 [==============================] - 70s 10ms/step - loss: 0.9929 - acc: 0.5156 - val_loss: 1.0243 - val_acc: 0

7000/7000 [==============================] - 69s 10ms/step - loss: 0.7156 - acc: 0.6907 - val_loss: 2.4386 - val_acc: 0.3625
Epoch 61/100
7000/7000 [==============================] - 69s 10ms/step - loss: 0.7109 - acc: 0.6894 - val_loss: 2.1084 - val_acc: 0.2825
Epoch 62/100
7000/7000 [==============================] - 69s 10ms/step - loss: 0.7095 - acc: 0.6837 - val_loss: 1.5623 - val_acc: 0.5450
Epoch 63/100
7000/7000 [==============================] - 69s 10ms/step - loss: 0.7020 - acc: 0.6904 - val_loss: 2.2618 - val_acc: 0.2675
Epoch 64/100
7000/7000 [==============================] - 69s 10ms/step - loss: 0.6968 - acc: 0.6947 - val_loss: 5.3948 - val_acc: 0.5950
Epoch 65/100
7000/7000 [==============================] - 69s 10ms/step - loss: 0.6921 - acc: 0.6946 - val_loss: 2.2103 - val_acc: 0.6025
Epoch 66/100
7000/7000 [==============================] - 69s 10ms/step - loss: 0.6950 - acc: 0.6976 - val_loss: 1.9378 - val_acc: 0.5325
Epoch 67/100
7000/7000 [=======================

In [28]:
print(len(model.layers))

13


In [12]:
input_img = Input(shape=(128, 128, 3))
x = Conv2D(16, (7, 7), padding='same')(input_img)
#x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(32, (5, 5), padding='same')(x)
#x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(64, (3, 3), padding='same')(x)
#x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Flatten()(x)
Softmax = Dense(3, activation = "softmax")(x)

model = Model(input_img,Softmax)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 128, 128, 16)      2368      
_________________________________________________________________
activation_16 (Activation)   (None, 128, 128, 16)      0         
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 64, 64, 16)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 64, 64, 32)        12832     
_________________________________________________________________
activation_17 (Activation)   (None, 64, 64, 32)        0         
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 32, 32, 32)        0         
__________

In [7]:
input_img = Input(shape=(299, 299, 3))
x = Conv2D(4, (7, 7), padding='same')(input_img)
#x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (5, 5), padding='same')(x)
#x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(16, (3, 3), padding='same')(x)
#x = BatchNormalization()(x)
x = Activation('relu')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)
x = Flatten()(encoded)
x = Dense(256)(x)
x = Activation("relu")(x)
Softmax = Dense(3, activation = "softmax")(x)

model = Model(input_img,Softmax)

model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=["accuracy"])
hist = model.fit(x_train,train_label_1,
                 batch_size=64,
                    nb_epoch=300,
                    validation_data=(x_test,test_label_1),
                    shuffle=True)


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:26: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 9000 samples, validate on 1500 samples
Epoch 1/300
9000/9000 [==============================] - 10s 1ms/step - loss: 1.1000 - acc: 0.3323 - val_loss: 1.0985 - val_acc: 0.3433
Epoch 2/300
9000/9000 [==============================] - 9s 1ms/step - loss: 1.0987 - acc: 0.3369 - val_loss: 1.0986 - val_acc: 0.3333
Epoch 3/300
3264/9000 [=========>....................] - ETA: 5s - loss: 1.0977 - acc: 0.3652

KeyboardInterrupt: 

In [ ]:

keras.applications.inception_v3.InceptionV3(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)